In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from PIL import Image
import numpy as np

train_data_dir = r"C:\Users\Ayush\OneDrive\Desktop\skin2\train"
validation_data_dir = r"C:\Users\Ayush\OneDrive\Desktop\skin2\train"
img_width, img_height = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

model = Sequential()
model.add(Conv2D(1024, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(230, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(200, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(180, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(23, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy')

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

model.save('skin_disease_classifier.h5')

model = load_model('skin_disease_classifier.h5')

image_path = r"C:\Users\Ayush\OneDrive\Desktop\skin\Dataset\train\Vasculitis Photos\stevens-johnson-syndrome-5.jpg"

def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

new_image = preprocess_image(image_path)

predictions = model.predict(new_image)

predicted_class_index = np.argmax(predictions)

class_labels = [
    "Acne And Rosacea", "Actinic Keratosis", "Atopic Dermatitis", "Bullous Disease",
    "Cellulitis Impetigo", "Eczema", "Exanthems and Drug Eruptions", "HPV",
    "Light Disease", "Lupus", "Melanoma", "Poison Ivy", "Psoriasis", "Seborrheic Keratoses",
    "Systematic Disease", "Tinea Ringworm Candidiasis", "Urticaria Hives",
    "Vascular Tumors", "Warts Molluscum"
]

predicted_class_label = class_labels[predicted_class_index]

predicted_probability = predictions[0][predicted_class_index]

print(f"Predicted class: {predicted_class_label}")
print(f"Probability: {predicted_probability:.2f}")
